In [ ]:
pip install mysql-connector-python

In [ ]:
import pandas as pd
import numpy as np
import importlib.metadata
import mysql.connector
import streamlit as st

from datetime import datetime

In [128]:
# Cargar csv
df_combustibles = pd.read_csv("../bin/listado_combustibles.csv")
df_concesionarios = pd.read_csv("../bin/listado_concesionarios.csv")
df_distintivo_ambiental = pd.read_csv("../bin/listado_distintivo_ambiental.csv")
df_marcas = pd.read_csv("../bin/listado_marcas.csv")
df_modelos = pd.read_csv("../bin/listado_modelos.csv")
df_provincias = pd.read_csv("../bin/listado_provincias.csv")
df_sobrealimentacion = pd.read_csv("../bin/listado_sobrealimentacion.csv")
df_traccion = pd.read_csv("../bin/listado_traccion.csv")
df_prestaciones = pd.read_csv("../bin/prestaciones.csv")
df_combustibles

,id_combustible,nombre_combustible
0,1,Diesel
1,2,Gasolina
2,3,Híbrido Enchufable
3,4,Eléctrico
4,5,Gasolina/gas
5,6,NaN
6,7,Gas


In [ ]:
#Comprobar si hay nans
df_distintivo_ambiental.isna().sum()

In [130]:
#Convertir a nan a none
for columna in df_combustibles.columns:
    df_combustibles[columna] = df_combustibles[columna].replace([np.nan, pd.NA], None)

In [132]:
df_combustibles

,id_combustible,nombre_combustible
0,1,Diesel
1,2,Gasolina
2,3,Híbrido Enchufable
3,4,Eléctrico
4,5,Gasolina/gas
5,6,None
6,7,Gas


In [136]:
#Eliminar nans
df_combustibles = df_combustibles.dropna(subset=['nombre_combustible'])

In [138]:
df_combustibles

,id_combustible,nombre_combustible
0,1,Diesel
1,2,Gasolina
2,3,Híbrido Enchufable
3,4,Eléctrico
4,5,Gasolina/gas
6,7,Gas


In [ ]:
# Conectar a la base de datos usando los valores almacenados en `secrets.toml`
def conectar_base_datos():
    conn = mysql.connector.connect(
        host=st.secrets["database"]["host"],
        user=st.secrets["database"]["user"],
        password=st.secrets["database"]["password"],
        database=st.secrets["database"]["database"]
    )
    return conn

In [134]:
# Funcion para insertar coches en la base de datos
def insertar_datos(df, nombre_tabla):
    # Conexión a la base df_cochedatos de MySQL
    conn = mysql.connector.connect(
        host = "localhost",
        user = "root",
        password = "Whereisbotus1996",
        database = "coches_segunda_mano"
    )
    # Crear un cursor para ejecutar comandos SQL
    cursor = conn.cursor()
    # Obtener las columnas del DataFrame
    columnas = df.columns.tolist()
    # Convertir cada fila del DataFrame a una tupla
    valores = [tuple(row) for row in df.to_numpy()]

    # Crear la consulta SQL para insertar datos
    query = f"""
        INSERT INTO {nombre_tabla} ({', '.join(columnas)})
        VALUES ({', '.join(['%s'] * len(columnas))})
    """

    # Ejecutar la consulta para todas las filas
    cursor.executemany(query, valores)

    # Confirmar los cambios en la base de datos
    conn.commit()

    # Cerrar el cursor y la conexion
    cursor.close()
    conn.close()

    # Mensaje de exito
    print("Datos insertados correctamente")

# Llamar a la funcion para insertar los datos en la tabla
insertar_datos(df_combustibles, 'combustibles')

IntegrityError: 1048 (23000): Column 'nombre_combustible' cannot be null

In [ ]:
# Funcion para extraer datos
def extraer_datos(nombre_tabla):
    # Conexión a la base de datos de MySQL
    conn = mysql.connector.connect(
        host = "localhost",
        user = "root",
        password = "Whereisbotus1996",
        database = "coches_segunda_mano"
    )
    # Crear un cursor para ejecutar comandos SQL
    cursor = conn.cursor()

    # Crear la consulta SQL para seleccionar todos los datos de la tabla
    query = f"SELECT * FROM {nombre_tabla}"

    # Ejecutar la consulta y recuperar los datos
    cursor.execute(query)
    resultado = cursor.fetchall()

    # Obtener los nombres de las columnas de la tabla
    columnas = [desc[0] for desc in cursor.description]

    # Convertir los resultados a un DataFrame
    df = pd.DataFrame(resultado, columns=columnas)

    # Cerrar el cursor y la conexión
    cursor.close()
    conn.close()

    # Devolver el DataFrame
    return df

# Llamar a la función para extraer datos de la tabla
df_extraido = extraer_datos('tipo_traccion')
print(df_extraido)